In [1]:
from sklearn.model_selection import GridSearchCV
import numpy as np

X = np.load('data/X_train.npy')#[0:load_n]
y = np.genfromtxt('data/y_1.csv', delimiter='\n')#[0:load_n]
#data_test = np.load('data/X_test.npy')


In [12]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.base import BaseEstimator, TransformerMixin
import random
random.seed(99)

class GivenBinHistExtraction(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_new = []
        X_new = X
        return X_new

class ClusteredHistExtraction(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, n_samples=3, images_x_from=False, images_x_to=False):
        self.n_clusters = n_clusters
        self.n_samples = n_samples
        self.images_x_from = images_x_from
        self.images_x_to = images_x_to


    def cutImage(self, x):
        if self.images_x_from is not False and self.images_x_to is not False:
            #images = np.split(row, 176)[50:130] # pretty optimal already
            side_images = np.split(x, 176)[self.images_x_from : self.images_x_to]
            x = np.array(side_images).flatten()
        return x


    def fit(self, X, y=None):
        samples = random.sample(list(X), self.n_samples)
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_jobs=-1, random_state=42)
        # self.kmeans = MiniBatchKMeans(n_clusters=self.n_clusters, batch_size=100, random_state=42)

        centers = []
        for i, sample in enumerate(samples):
            # sample = sample[1672390 : -786303]
            sample = self.cutImage(sample)

            sample = sample[(sample > 0) & (sample < 1800)]
            self.kmeans.fit(np.array([sample]).T)
            centers.append(np.sort(np.array(self.kmeans.cluster_centers_).flatten()))
            
            samples[i] = sample
            #print(str(i) + ' done')

        if True: # use all centers
            values = np.array(centers).flatten()
            values = np.sort(values)
        else: # take means of centers
            values = np.mean(centers, axis=0)

        # compute cluster centers
        #self.kmeans.fit(np.array(samples).T)
        #values = self.kmeans.cluster_centers_.T
        #print('fitted')

        # mean of the clusters over the rows
        #for i, v in enumerate(values.T):
        #    values.T[i] = np.sort(v)

        #values = np.mean(values.T, axis=0)

        self.edges = [1] # leave out 0
        for center_1, center_2 in zip(values[:-1], values[1:]):
            self.edges.append(.5 * (center_1 + center_2))

        print('n edges: ' + str(len(self.edges)))
        return self

    def transform(self, X, y=None):
        # np.histogram to make bins from edges, counts the number of pixels
        X_new = []
        for x in X:
            x = self.cutImage(x)
            x = x[(x > 0) & (x < 1800)]
            hist = np.histogram(x, bins=self.edges)
            X_new.append(hist[0])

        return X_new
    
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.kernel_ridge import KernelRidge

pipe = Pipeline([
    ('ClusteredHistExtraction', ClusteredHistExtraction(
        n_clusters=8,n_samples=3)),
    ('scaler', StandardScaler()),
    ('vct', VarianceThreshold()),
    ('kernelRidge', KernelRidge(kernel='polynomial'))
])

In [13]:
from sklearn.model_selection import GridSearchCV

parameters = {
 #   'ClusteredHistExtraction__n_clusters': [8, 10],
 #   'ClusteredHistExtraction__n_samples': [3, 4]
}
grid = GridSearchCV(pipe, parameters, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(str(np.mean(grid.cv_results_['mean_test_score'])) + ', ' + str(np.mean(grid.cv_results_['std_test_score'])))

# -68.8529277579, 5.72349220142
# -68.5395679288, 6.39975328753

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


0 done
0 done
0 done
0 done
1 done
1 done
1 done
1 done
2 done
n edges: 24
2 done
n edges: 24
2 done
n edges: 24
2 done
3 done
n edges: 32


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


0 done
0 done
0 done


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


1 done
1 done
1 done
2 done
0 done
2 done
3 done
n edges: 32
2 done
n edges: 30
3 done
n edges: 32
1 done
2 done
n edges: 30


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


0 done
0 done
0 done


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


1 done
1 done
1 done
0 done
2 done
2 done
n edges: 30
2 done
1 done
3 done
n edges: 40
3 done
n edges: 40
2 done
3 done
n edges: 40
0 done
1 done
2 done
n edges: 24
-68.5395679288, 6.39975328753


In [17]:
grid.cv_results_
#grid.best_params_

{'ClusteredHistExtraction__n_clusters': 8,
 'ClusteredHistExtraction__n_samples': 3}

In [20]:
# Search bin subdivision so that there is highest variance
# 1. Divide hist into bins over some rows
# 2. Compute std -> divide bins where it's high, join/drop those where it's low
# 3. Repeat / change rows
kmeans = KMeans(n_clusters=8, n_jobs=-1, random_state=42)

samples = X[0: 5]
centers = []
for i, sample in enumerate(samples):
    # sample = sample[1672390 : -786303]

    sample = sample[(sample > 0) & (sample < 1800)]
    kmeans.fit(np.array([sample]).T)
    centers.append(np.sort(np.array(kmeans.cluster_centers_).flatten()))

    #samples[i] = sample
    #print(str(i) + ' done')

if True: # use all centers
    values = np.array(centers).flatten()
    values = np.sort(values)
else: # take means of centers
    values = np.mean(centers, axis=0)

edges = [1] # leave out 0
for center_1, center_2 in zip(values[:-1], values[1:]):
    edges.append(.5 * (center_1 + center_2))

print('n edges: ' + str(len(edges)))
edges

n edges: 40


[1,
 268.59153353677925,
 281.00824805433393,
 296.51672876785585,
 312.41929109857142,
 385.23831822211014,
 458.12983660320458,
 472.33076634277745,
 493.82267048266544,
 510.43647791541645,
 576.57399025095651,
 642.540938664245,
 648.54789979548752,
 655.08349269468147,
 666.54241844275282,
 724.33201508885827,
 779.89739389945362,
 783.20843221048995,
 785.82438478516588,
 807.58109023044335,
 869.72060524581752,
 913.23145834367801,
 915.91949110352323,
 919.63925603546056,
 950.7720301599602,
 1021.6951866283912,
 1069.5507893466106,
 1076.2284155454829,
 1078.4194846165499,
 1107.9840749772698,
 1178.3353557602009,
 1228.4902326692686,
 1237.3030234970834,
 1243.0805123468485,
 1277.8951157600625,
 1339.3753602258789,
 1372.8570616285097,
 1379.1746465921135,
 1388.0591113102987,
 1433.3192436369491]

In [99]:
from sklearn.utils import resample

edges = [1,
 160,
 180,
 190.0,
 200,
 210,
 220,
 230.0,
 240,
 250.29576676838963,
 260.59153353677925,
 270.7998907955566,
 281.00824805433393,
 296.51672876785585,
 312.4192910985714,
 328.7096455492857,
 345,
 365.11915911105507,
 385.23831822211014,
 403.4611978173838,
 421.68407741265736,
 439.90695700793094,
 458.1298366032046,
 484.28315725931054,
 510.43647791541645,
 576.5739902509565,
 609.5574644576008,
 626.0492015609229,
 642.540938664245,
 655.0834926946815,
 666.5424184427528,
 673.2712092213765,
 680,
 685,
 690,
 697.5830037722146,
 705.1660075444291,
 712.7490113166436,
 720.3320150888583,
 725.2490113166436,
 730.1660075444291,
 735.0830037722146,
 740,
 747.7243484748634,
 755.4486969497268,
 763.1730454245902,
 770.8973938994536,
 778.3608893423097,
 785.8243847851659,
 791.9121923925829,
 798,
 807.5810902304434,
 816.5985296073651,
 825.6159689842868,
 834.6334083612087,
 843.6508477381304,
 861.685726491974,
 879.7206052458175,
 896.4760317947478,
 913.231458343678,
 919.6392560354606,
 935.2056430977104,
 950.7720301599602,
 968.5028192770679,
 986.2336083941757,
 1003.9643975112834,
 1021.6951866283912,
 1045.622987987501,
 1069.5507893466106,
 1096.7469309500082,
 1123.9430725534057,
 1151.1392141568033,
 1178.3353557602009,
 1203.4127942147347,
 1228.4902326692686,
 1240.8414534419671,
 1253.1926742146657,
 1265.5438949873642,
 1277.8951157600625,
 1288.9475578800311,
 1300,
 1309.8438400564696,
 1319.6876801129395,
 1329.5315201694093,
 1339.375360225879,
 1347.7457855765365,
 1356.1162109271943,
 1364.486636277852,
 1372.8570616285097,
 1379.6427962213822,
 1386.4285308142548,
 1393.2142654071274,
 1400,
 1408.3298109092373,
 1416.6596218184745,
 1424.9894327277118,
 1433.319243636949,
 1442,
 1448.5,
 1455,
 1461.25,
 1467.5,
 1473.75,
 1480,
 1490.0,
 1500,
 1507.5,
 1515,
 1522.5,
 1530,
 1540.0,
 1550,
 1562.5,
 1575.0,
 1600,
 1650.0,
 2000]
hists = []
for x in X[0:150]:#resample(X, n_samples=100):
    hists.append(np.histogram(x, bins=edges)[0])
    
std_all = np.std(np.array(hists).flatten())
for bin_i in range(len(hists[0])):
    arr = [h[bin_i] for h in hists]
    print("bin "+str(bin_i)+": "+str(np.std(arr) / std_all))

bin 0: 0.19288097836
bin 1: 0.386191795817
bin 2: 0.325874415311
bin 3: 0.41222737076
bin 4: 0.47542922106
bin 5: 0.506458292252
bin 6: 0.506221216049
bin 7: 0.483008357662
bin 8: 0.486625832258
bin 9: 0.399419555859
bin 10: 0.358520549664
bin 11: 0.355890166316
bin 12: 0.43180574199
bin 13: 0.406719502854
bin 14: 0.364685406343
bin 15: 0.330587735671
bin 16: 0.395664265317
bin 17: 0.344734717995
bin 18: 0.285880477721
bin 19: 0.266985103695
bin 20: 0.246351854336
bin 21: 0.23865836085
bin 22: 0.28940555062
bin 23: 0.248759762034
bin 24: 0.479587062477
bin 25: 0.401732418664
bin 26: 0.316755989561
bin 27: 0.385834622373
bin 28: 0.386101622476
bin 29: 0.383571589285
bin 30: 0.270758737062
bin 31: 0.249409890169
bin 32: 0.218499436158
bin 33: 0.22912181373
bin 34: 0.388013703577
bin 35: 0.41230981479
bin 36: 0.374679925066
bin 37: 0.445536217218
bin 38: 0.285757314955
bin 39: 0.292428613351
bin 40: 0.293164239758
bin 41: 0.236597422167
bin 42: 0.474170239472
bin 43: 0.473736087251
bin 44

In [100]:
from IPython.display import display
new_edges = edges # attention: 49 bins = 50 edges
for i in range(3):
    # compute hists for samples
    hists = []
    for x in resample(X, n_samples=200):
        hists.append(np.histogram(x, bins=new_edges)[0])

    ed = [1]
    for bin_i in range(len(hists[0])):
        # compute std for bin i
        arr = [h[bin_i] for h in hists]
        std = np.std(arr) / std_all
        
        if std > 0.4: # if std over threshold, split bin
            ed.append(0.5 * (new_edges[bin_i] + new_edges[bin_i+1])) 
            
        ed.append(new_edges[bin_i+1])
        
    new_edges = ed
    if len(new_edges)>=120: break
        
print(len(new_edges))
display(new_edges)

117


[1,
 160,
 180,
 190.0,
 200,
 210,
 220,
 230.0,
 240,
 250.29576676838963,
 260.59153353677925,
 270.7998907955566,
 281.00824805433393,
 296.51672876785585,
 312.4192910985714,
 328.7096455492857,
 345,
 365.11915911105507,
 385.23831822211014,
 403.4611978173838,
 421.68407741265736,
 439.90695700793094,
 458.1298366032046,
 484.28315725931054,
 510.43647791541645,
 576.5739902509565,
 609.5574644576008,
 626.0492015609229,
 642.540938664245,
 655.0834926946815,
 666.5424184427528,
 673.2712092213765,
 680,
 685,
 690,
 697.5830037722146,
 705.1660075444291,
 712.7490113166436,
 720.3320150888583,
 725.2490113166436,
 730.1660075444291,
 735.0830037722146,
 740,
 747.7243484748634,
 755.4486969497268,
 763.1730454245902,
 770.8973938994536,
 778.3608893423097,
 785.8243847851659,
 791.9121923925829,
 798,
 807.5810902304434,
 816.5985296073651,
 825.6159689842868,
 834.6334083612087,
 843.6508477381304,
 861.685726491974,
 879.7206052458175,
 896.4760317947478,
 913.231458343678,
 